# Notebook 8: Plant Biology & Agricultural Genomics

**From crop genomes to field phenotypes -- computational approaches to food security**

Prerequisites: Notebooks 1-7 (sequences, genomics, transcriptomics, protein structure, RNAseq, clinical, imaging)

This notebook builds:
1. Plant genome characteristics (polyploidy, TE content, genome size variation)
2. Crop domestication genetics (selective sweeps, bottleneck signatures)
3. GWAS for agricultural traits (yield, drought tolerance, disease resistance)
4. QTL mapping concepts
5. Marker-assisted selection (MAS) simulation
6. Phenotype prediction from genomic data
7. Environmental x Genotype interaction (GxE)
8. Crop diversity and conservation genetics

**Data sources**: Curated crop genome statistics (15 species with genome sizes, ploidy, TE content). Arabidopsis 1001 Genomes-derived SNP data (200 accessions x 1000 SNPs across 5 chromosomes) with flowering time phenotypes. GxE interaction and diversity timeline kept as simulations (real multi-environment trial data not freely available).

**Data setup**: Run `python data/download_all_data.py` from the `Compute/` directory before executing.

Estimated runtime: ~3 minutes

**Key learning outcomes:**
1. Understand how plant genomes differ from animal genomes -- see [[Plant-Animal Divergence]]
2. Simulate and detect domestication bottlenecks and selective sweeps -- see [[Fitness Landscapes]]
3. Run GWAS on crop breeding lines and interpret Manhattan plots -- see [[Variant-Phenotype Mapping]]
4. Build genomic prediction models for yield -- see [[Genotype-to-Phenotype Hub]]
5. Analyze genotype-by-environment interaction -- see [[Context Conditionality]]
6. Quantify genetic erosion and the role of conservation -- see [[Robustness and Evolvability]]

---

## Section 0: Setup

We use the standard scientific Python stack: numpy for simulation, scipy for statistics, sklearn for genomic prediction, pandas for data handling, and matplotlib for visualization. No specialized plant biology libraries are required.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
print("Ready -- numpy, scipy, sklearn, pandas, matplotlib")

---

## Section 1: Plant Genome Characteristics

Plant genomes are dramatically different from animal genomes:

- **Polyploidy**: Many crops are polyploid (wheat = hexaploid 6x, potato = tetraploid 4x)
- **Genome size**: Ranges from 63 Mb (Genlisea) to 150 Gb (Paris japonica) -- 2400x variation!
- **Transposable elements**: Often >80% of genome (maize ~85%)
- **Gene duplication**: Whole-genome duplications (WGD) are common

Reference [[Degeneracy in Biological Systems]] -- polyploidy creates massive functional redundancy. Multiple copies of every gene means the system can tolerate mutations, explore new functions, and buffer against perturbation.

$$\text{Wheat genome: } 3 \text{ subgenomes} \times 7 \text{ chromosomes} \times 2 \text{ (diploid)} = 42 \text{ chromosomes}$$

### Genome size comparison

In [ ]:
# Load real crop genome statistics (15 species)
crop_df = pd.read_csv('data/nb08/crop_genome_stats.csv')
print(f"Loaded genome stats for {len(crop_df)} species")
print(crop_df[['common_name', 'genome_size_mb', 'ploidy', 'te_percent']].to_string(index=False))

# Bar chart of genome sizes (log scale) colored by ploidy
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

colors = {2: '#2196F3', 3: '#FF9800', 4: '#4CAF50', 6: '#F44336'}
bar_colors = [colors[p] for p in crop_df['ploidy']]

axes[0].barh(crop_df['common_name'], crop_df['genome_size_mb'], color=bar_colors)
axes[0].set_xscale('log')
axes[0].set_xlabel('Genome Size (Mb, log scale)')
axes[0].set_title('Plant Genome Sizes')
# Add ploidy legend
for ploidy_val, col in colors.items():
    axes[0].barh([], [], color=col, label=f'{ploidy_val}x')
axes[0].legend(title='Ploidy', loc='lower right')

# Gene count vs genome size
axes[1].scatter(crop_df['genome_size_mb'], crop_df['gene_count'],
               c=[colors[p] for p in crop_df['ploidy']], s=100, edgecolors='black')
for _, row in crop_df.iterrows():
    axes[1].annotate(row['common_name'], (row['genome_size_mb'], row['gene_count']),
                    fontsize=7, ha='left', va='bottom')
axes[1].set_xscale('log')
axes[1].set_xlabel('Genome Size (Mb)')
axes[1].set_ylabel('Gene Count')
axes[1].set_title('Gene Count vs Genome Size')

plt.tight_layout()
plt.show()

# Key insight: genome size does NOT predict gene count
r, p = stats.pearsonr(np.log10(crop_df['genome_size_mb']), crop_df['gene_count'])
print(f"\nCorrelation (log genome size vs gene count): r={r:.3f}, p={p:.3f}")
print("Wheat has 17 Gb but only ~3x genes of 135 Mb Arabidopsis -- most is TEs!")

### Transposable element content

In [ ]:
# Transposable element content from real crop genome data
te_data = crop_df[['common_name', 'te_percent']].sort_values('te_percent')

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.barh(te_data['common_name'], te_data['te_percent'], color='#FF7043', label='TE content')
ax.barh(te_data['common_name'], 100 - te_data['te_percent'],
        left=te_data['te_percent'], color='#66BB6A', label='Genes + Other')
ax.set_xlabel('Genome Composition (%)')
ax.set_title('Transposable Element Content Across Crop Species')
ax.legend(loc='lower right')
ax.set_xlim(0, 100)

plt.tight_layout()
plt.show()

print(f"Highest TE content: {te_data.iloc[-1]['common_name']} ({te_data.iloc[-1]['te_percent']}%)")
print(f"Lowest TE content:  {te_data.iloc[0]['common_name']} ({te_data.iloc[0]['te_percent']}%)")

### Gene family expansion through WGD

In [ ]:
# Gene family expansion through whole-genome duplication (WGD)
# Model using real Arabidopsis gene count as starting point
arabidopsis_genes = crop_df[crop_df['common_name'] == 'Arabidopsis']['gene_count'].values[0]

# Simulate WGD -> fractionation -> tandem duplication
np.random.seed(42)
stages = ['Ancestral', 'WGD (2x)', 'Fractionation', 'Tandem Dup']
gene_counts = [arabidopsis_genes]
gene_counts.append(arabidopsis_genes * 2)              # WGD doubles everything
gene_counts.append(int(arabidopsis_genes * 2 * 0.65))  # ~35% gene loss (fractionation)
gene_counts.append(int(gene_counts[-1] * 1.05))        # ~5% gain from tandem duplications

# Simulate duplicate pair divergence over time (MY = million years)
time_points = np.linspace(0, 100, 200)  # 0-100 MY after WGD
# Sequence identity decays exponentially; some duplicates neofunctionalize
identity = 100 * np.exp(-0.01 * time_points) + np.random.normal(0, 2, len(time_points))
neofunc_prob = 1 - np.exp(-0.005 * time_points)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(stages, gene_counts, color=['#90CAF9', '#F44336', '#FF9800', '#4CAF50'])
axes[0].set_ylabel('Gene Count')
axes[0].set_title('Gene Count Through WGD Cycle')
for i, v in enumerate(gene_counts):
    axes[0].text(i, v + 500, f'{v:,}', ha='center', fontsize=10)

axes[1].plot(time_points, identity, color='#2196F3', alpha=0.7, label='Sequence identity (%)')
ax2 = axes[1].twinx()
ax2.plot(time_points, neofunc_prob * 100, color='#F44336', label='Neofunctionalization (%)')
axes[1].set_xlabel('Time After WGD (Million Years)')
axes[1].set_ylabel('Sequence Identity (%)', color='#2196F3')
ax2.set_ylabel('Neofunctionalization Probability (%)', color='#F44336')
axes[1].set_title('Duplicate Gene Divergence Over Time')
axes[1].legend(loc='center left')
ax2.legend(loc='center right')

plt.tight_layout()
plt.show()

print(f"Starting genes (Arabidopsis): {arabidopsis_genes:,}")
print(f"After WGD: {gene_counts[1]:,} (2x duplication)")
print(f"After fractionation: {gene_counts[2]:,} (~35% loss)")
print(f"After tandem dup: {gene_counts[3]:,} (~5% gain)")

---

## Section 2: Crop Domestication Genetics

Domestication transformed wild plants into crops through intense artificial selection. Genetic signatures include:

- **Selective sweeps**: Reduced diversity near selected genes
- **Bottleneck**: Loss of diversity during domestication
- **Linkage drag**: Unwanted genes hitchhiking with selected alleles

Key domestication genes:
- **tb1** (maize): branching architecture (teosinte -> maize)
- **Sh1** (rice): shattering (seeds stay on plant)
- **Q** (wheat): free-threshing grain

Reference [[Fitness Landscapes]] -- domestication navigates the fitness landscape under human-defined objectives rather than natural selection.

### Domestication bottleneck simulation

In [ ]:
# Load real Arabidopsis 1001 Genomes SNP data
snp_df = pd.read_csv('data/nb08/arabidopsis_snps.csv', index_col='accession_id')
pheno_df = pd.read_csv('data/nb08/arabidopsis_phenotypes.csv', index_col='accession_id')
print(f"Genotype matrix: {snp_df.shape[0]} accessions x {snp_df.shape[1]} SNPs")
print(f"Phenotype data: {pheno_df.shape[0]} accessions")
print(f"Geographic groups: {pheno_df['geographic_group'].value_counts().to_dict()}")

# Compute allele frequencies across all accessions
allele_freqs = snp_df.mean() / 2  # Convert 0/1/2 to allele freq (0-1)

# Simulate domestication bottleneck using real allele frequencies as "wild"
np.random.seed(42)
n_loci = 200
wild_freqs = allele_freqs.values[:n_loci]

# Bottleneck: sample 20 plants (40 alleles) from wild population
bottleneck_size = 20
bottleneck_freqs = np.array([
    np.random.binomial(2 * bottleneck_size, f) / (2 * bottleneck_size)
    for f in wild_freqs
])

# Modern elite: further bottleneck from 50 plants
elite_size = 50
elite_freqs = np.array([
    np.random.binomial(2 * elite_size, f) / (2 * elite_size)
    for f in bottleneck_freqs
])

# Compute expected heterozygosity: He = 2p(1-p)
He_wild = np.mean(2 * wild_freqs * (1 - wild_freqs))
He_bottleneck = np.mean(2 * bottleneck_freqs * (1 - bottleneck_freqs))
He_elite = np.mean(2 * elite_freqs * (1 - elite_freqs))

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, freqs, title in zip(axes, [wild_freqs, bottleneck_freqs, elite_freqs],
                             ['Wild (Arabidopsis)', 'Bottleneck (n=20)', 'Elite (n=50)']):
    ax.hist(freqs, bins=20, range=(0, 1), color='#66BB6A', edgecolor='black', alpha=0.7)
    He = np.mean(2 * freqs * (1 - freqs))
    ax.set_title(f"{title}\nHe = {He:.3f}")
    ax.set_xlabel('Allele Frequency')
    ax.set_ylabel('Count')

plt.suptitle('Domestication Bottleneck: Progressive Loss of Genetic Diversity', fontsize=14)
plt.tight_layout()
plt.show()

print(f"Expected heterozygosity (He):")
print(f"  Wild:       {He_wild:.4f}")
print(f"  Bottleneck: {He_bottleneck:.4f} ({100*(He_bottleneck-He_wild)/He_wild:+.1f}%)")
print(f"  Elite:      {He_elite:.4f} ({100*(He_elite-He_wild)/He_wild:+.1f}%)")

### Selective sweep signature

In [ ]:
# Selective sweep signature around a domestication gene
# Use real chr1 SNP positions to simulate sweep on real genomic coordinates
chr1_cols = [c for c in snp_df.columns if c.startswith('chr1_')]
chr1_positions = np.array([int(c.split('_')[1]) for c in chr1_cols])

# Compute real diversity (He) at each chr1 locus
chr1_he = np.array([2 * (snp_df[c].mean()/2) * (1 - snp_df[c].mean()/2) for c in chr1_cols])

# Simulate a selective sweep centered on a "domestication gene" at position ~15 Mb
sweep_center = 15_000_000
sweep_width = 2_000_000  # 2 Mb sweep region
distance_from_sweep = np.abs(chr1_positions - sweep_center)
sweep_reduction = np.exp(-0.5 * (distance_from_sweep / sweep_width) ** 2)
swept_he = chr1_he * (1 - 0.9 * sweep_reduction)  # Up to 90% reduction at center

fig, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)

axes[0].scatter(chr1_positions / 1e6, chr1_he, s=10, alpha=0.5, color='#2196F3')
axes[0].set_ylabel('Expected Heterozygosity (He)')
axes[0].set_title('Wild-Type Diversity (Real Arabidopsis Chr1)')
axes[0].axhline(np.mean(chr1_he), color='red', linestyle='--', alpha=0.5, label=f'Mean He={np.mean(chr1_he):.3f}')
axes[0].legend()

axes[1].scatter(chr1_positions / 1e6, swept_he, s=10, alpha=0.5, color='#FF7043')
axes[1].axvline(sweep_center / 1e6, color='red', linestyle='--', label='Domestication gene')
axes[1].axvspan((sweep_center - sweep_width) / 1e6, (sweep_center + sweep_width) / 1e6,
               alpha=0.1, color='red', label='Sweep region')
axes[1].set_ylabel('Expected Heterozygosity (He)')
axes[1].set_xlabel('Chromosome 1 Position (Mb)')
axes[1].set_title('After Selective Sweep (Simulated Domestication)')
axes[1].legend()

plt.suptitle('Selective Sweep Reduces Diversity Near Selected Gene', fontsize=14)
plt.tight_layout()
plt.show()

print(f"Mean He (wild): {np.mean(chr1_he):.4f}")
print(f"He at sweep center: {swept_he[np.argmin(distance_from_sweep)]:.4f}")
print(f"Sweep region: {(sweep_center - sweep_width)/1e6:.1f} - {(sweep_center + sweep_width)/1e6:.1f} Mb")

---

## Section 3: Agricultural GWAS

GWAS in crops identifies genomic regions associated with agronomic traits: yield, drought tolerance, disease resistance, grain quality. Unlike human GWAS, plant GWAS can leverage controlled crosses, replicated field trials, and known pedigrees.

Reference [[Variant-Phenotype Mapping]] and [[Genotype-to-Phenotype Hub]] -- the fundamental challenge of mapping genotype to phenotype is the same across species.

### Simulate crop GWAS (yield trait)

In [ ]:
# GWAS on real Arabidopsis data: flowering time ~ SNP genotypes
# Merge genotype and phenotype data
common_ids = snp_df.index.intersection(pheno_df.index)
X = snp_df.loc[common_ids].values  # Genotype matrix (n_accessions x n_snps)
y = pheno_df.loc[common_ids, 'flowering_time_days'].values

print(f"GWAS dataset: {X.shape[0]} accessions x {X.shape[1]} SNPs")
print(f"Phenotype (flowering time): mean={y.mean():.1f} days, std={y.std():.1f} days")
print(f"Range: {y.min():.1f} - {y.max():.1f} days")

# Compute heritability estimate from geographic group means
groups = pheno_df.loc[common_ids, 'geographic_group']
group_means = groups.map(pheno_df.loc[common_ids].groupby('geographic_group')['flowering_time_days'].mean())
ss_between = np.sum((group_means - y.mean()) ** 2)
ss_total = np.sum((y - y.mean()) ** 2)
h2_group = ss_between / ss_total
print(f"Variance explained by geographic group: {h2_group:.3f}")

### Manhattan plot and QTL detection

In [ ]:
# Single-marker GWAS: linear regression for each SNP
snp_names = snp_df.columns.tolist()
p_values = []
effects = []

for j in range(X.shape[1]):
    snp_col = X[:, j]
    # Skip monomorphic SNPs
    if snp_col.std() == 0:
        p_values.append(1.0)
        effects.append(0.0)
        continue
    slope, intercept, r, p, se = stats.linregress(snp_col, y)
    p_values.append(p)
    effects.append(slope)

p_values = np.array(p_values)
effects = np.array(effects)
log_p = -np.log10(p_values + 1e-300)

# Extract chromosome and position for Manhattan plot
chroms = np.array([int(s.split('_')[0].replace('chr', '')) for s in snp_names])
positions = np.array([int(s.split('_')[1]) for s in snp_names])

# Manhattan plot
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

chrom_colors = {1: '#2196F3', 2: '#FF9800', 3: '#4CAF50', 4: '#9C27B0', 5: '#F44336'}
cumulative_pos = np.zeros(len(positions))
chrom_offsets = {}
offset = 0
for c in sorted(np.unique(chroms)):
    mask = chroms == c
    chrom_offsets[c] = offset
    cumulative_pos[mask] = positions[mask] + offset
    offset = cumulative_pos[mask].max() + 5_000_000

for c in sorted(np.unique(chroms)):
    mask = chroms == c
    axes[0].scatter(cumulative_pos[mask], log_p[mask], s=8, alpha=0.6,
                   color=chrom_colors[c], label=f'Chr{c}')

threshold = -np.log10(0.05 / len(p_values))  # Bonferroni
axes[0].axhline(threshold, color='red', linestyle='--', label=f'Bonferroni (p={0.05/len(p_values):.1e})')
axes[0].set_ylabel('-log10(p-value)')
axes[0].set_title('Manhattan Plot: Flowering Time GWAS (Real Arabidopsis Data)')
axes[0].legend(loc='upper right', fontsize=8)
axes[0].set_xlabel('Genomic Position')

# QQ plot
expected = -np.log10(np.linspace(1/len(p_values), 1, len(p_values)))
observed = np.sort(log_p)[::-1][:len(expected)]
axes[1].scatter(np.sort(expected), np.sort(observed[:len(expected)]), s=8, alpha=0.5)
axes[1].plot([0, max(expected)], [0, max(expected)], 'r--', label='Expected under null')
axes[1].set_xlabel('Expected -log10(p)')
axes[1].set_ylabel('Observed -log10(p)')
axes[1].set_title('QQ Plot')
axes[1].legend()

plt.tight_layout()
plt.show()

n_sig = (p_values < 0.05 / len(p_values)).sum()
print(f"Bonferroni threshold: p < {0.05/len(p_values):.2e}")
print(f"Significant SNPs: {n_sig}")
if n_sig > 0:
    sig_idx = np.where(p_values < 0.05 / len(p_values))[0]
    for idx in sig_idx[:10]:
        print(f"  {snp_names[idx]}: p={p_values[idx]:.2e}, effect={effects[idx]:.2f} days/allele")

### Genomic prediction (GBLUP-like)

In [ ]:
# Genomic prediction using Ridge regression (GBLUP-like)
# Predict flowering time from all SNPs simultaneously

# Center genotype matrix
X_centered = X - X.mean(axis=0)

# Cross-validated prediction accuracy
ridge = Ridge(alpha=100)
cv_scores = cross_val_score(ridge, X_centered, y, cv=5, scoring='r2')
print(f"5-fold CV R²: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Prediction accuracy vs training population size
train_sizes = [50, 80, 100, 120, 150, 180]
mean_r2 = []
std_r2 = []

for n_train in train_sizes:
    r2_scores = []
    for _ in range(20):  # 20 random train/test splits
        idx = np.random.permutation(len(y))
        X_train, X_test = X_centered[idx[:n_train]], X_centered[idx[n_train:]]
        y_train, y_test = y[idx[:n_train]], y[idx[n_train:]]
        if len(y_test) < 10:
            continue
        ridge.fit(X_train, y_train)
        r2 = ridge.score(X_test, y_test)
        r2_scores.append(max(r2, 0))  # Floor at 0
    mean_r2.append(np.mean(r2_scores))
    std_r2.append(np.std(r2_scores))

fig, ax = plt.subplots(figsize=(8, 5))
ax.errorbar(train_sizes, mean_r2, yerr=std_r2, marker='o', capsize=5, color='#2196F3')
ax.set_xlabel('Training Population Size')
ax.set_ylabel('Prediction R²')
ax.set_title('Genomic Prediction Accuracy vs Training Size')
ax.set_ylim(0, max(mean_r2) * 1.3 + 0.05)
ax.axhline(cv_scores.mean(), color='red', linestyle='--', alpha=0.5, label=f'Full CV R²={cv_scores.mean():.3f}')
ax.legend()

plt.tight_layout()
plt.show()

print(f"\nPrediction accuracy by training size:")
for n, r2, sd in zip(train_sizes, mean_r2, std_r2):
    print(f"  n={n}: R² = {r2:.3f} ± {sd:.3f}")

---

## Section 4: Genotype x Environment Interaction (GxE)

Genotype x Environment (GxE) interaction means a variety's performance depends on where it is grown. A drought-tolerant line may excel in dry environments but underperform in irrigated conditions.

Reference [[Context Conditionality]] -- f(genotype, environment) -> phenotype. The same genome produces different outcomes in different contexts.

$$Y_{ijk} = \mu + G_i + E_j + (GE)_{ij} + \epsilon_{ijk}$$

### GxE reaction norms

In [ ]:
# Genotype x Environment interaction simulation
# Use 8 varieties x 5 environments with realistic GxE patterns
np.random.seed(42)
n_varieties = 8
n_envs = 5
env_names = ['Drought', 'Low N', 'Standard', 'Irrigated', 'Optimal']
env_quality = np.array([-2, -1, 0, 1, 2])  # Environmental index

# Base genetic values for each variety
genetic_values = np.array([6.0, 5.5, 7.0, 5.0, 6.5, 7.5, 4.5, 6.8])

# GxE slopes (sensitivity to environment)
# Variety 0: drought specialist (low slope -- stable)
# Variety 5: responsive (high slope -- gains in good environments)
gxe_slopes = np.array([0.3, 0.8, 1.0, 0.5, 1.2, 1.5, 0.4, 0.9])

# Generate yield data
yields = np.zeros((n_varieties, n_envs))
for i in range(n_varieties):
    for j in range(n_envs):
        yields[i, j] = genetic_values[i] + gxe_slopes[i] * env_quality[j] + np.random.normal(0, 0.3)

# Reaction norm plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cmap = plt.cm.tab10
for i in range(n_varieties):
    axes[0].plot(env_quality, yields[i], 'o-', color=cmap(i), label=f'Var {i}', alpha=0.8)
axes[0].set_xlabel('Environmental Index')
axes[0].set_ylabel('Yield (t/ha)')
axes[0].set_title('Reaction Norms: GxE Interaction')
axes[0].set_xticks(env_quality)
axes[0].set_xticklabels(env_names, rotation=30)
axes[0].legend(fontsize=7, ncol=2)

# ANOVA-style variance decomposition
grand_mean = yields.mean()
ss_g = n_envs * np.sum((yields.mean(axis=1) - grand_mean) ** 2)
ss_e = n_varieties * np.sum((yields.mean(axis=0) - grand_mean) ** 2)
ss_ge = np.sum((yields - yields.mean(axis=1, keepdims=True) -
               yields.mean(axis=0, keepdims=True) + grand_mean) ** 2)
ss_total = np.sum((yields - grand_mean) ** 2)

components = [ss_g/ss_total, ss_e/ss_total, ss_ge/ss_total,
              1 - (ss_g + ss_e + ss_ge)/ss_total]
axes[1].pie(components, labels=['Genotype', 'Environment', 'GxE', 'Residual'],
           colors=['#2196F3', '#4CAF50', '#FF9800', '#BDBDBD'],
           autopct='%1.1f%%', startangle=90)
axes[1].set_title('Variance Decomposition')

plt.tight_layout()
plt.show()

print(f"Variance components:")
print(f"  Genotype:    {100*ss_g/ss_total:.1f}%")
print(f"  Environment: {100*ss_e/ss_total:.1f}%")
print(f"  GxE:         {100*ss_ge/ss_total:.1f}%")

### Finlay-Wilkinson stability analysis

In [ ]:
# Finlay-Wilkinson stability analysis
env_means = yields.mean(axis=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

slopes = []
intercepts = []
for i in range(n_varieties):
    slope, intercept, r, p, se = stats.linregress(env_means, yields[i])
    slopes.append(slope)
    intercepts.append(intercept)
    axes[0].scatter(env_means, yields[i], color=cmap(i), s=30)
    x_line = np.array([env_means.min(), env_means.max()])
    axes[0].plot(x_line, intercept + slope * x_line, '--', color=cmap(i), alpha=0.5)

axes[0].plot(env_means, env_means, 'k-', linewidth=2, alpha=0.3, label='Average response')
axes[0].set_xlabel('Environment Mean Yield (t/ha)')
axes[0].set_ylabel('Variety Yield (t/ha)')
axes[0].set_title('Finlay-Wilkinson Regression')

# Stability vs mean yield
variety_means = yields.mean(axis=1)
axes[1].scatter(variety_means, slopes, c=[cmap(i) for i in range(n_varieties)], s=100, edgecolors='black')
for i in range(n_varieties):
    axes[1].annotate(f'V{i}', (variety_means[i], slopes[i]),
                    fontsize=9, ha='left', va='bottom')
axes[1].axhline(1.0, color='red', linestyle='--', alpha=0.5, label='Average stability (b=1)')
axes[1].set_xlabel('Mean Yield (t/ha)')
axes[1].set_ylabel('FW Slope (b)')
axes[1].set_title('Stability vs Mean Performance')
axes[1].legend()

plt.tight_layout()
plt.show()

print("Finlay-Wilkinson stability:")
for i in range(n_varieties):
    stability = "stable" if slopes[i] < 0.8 else "responsive" if slopes[i] > 1.2 else "average"
    print(f"  Var {i}: mean={variety_means[i]:.2f}, slope={slopes[i]:.2f} ({stability})")

---

## Section 5: Crop Diversity and Conservation Genetics

Modern breeding progressively narrows the genetic base of crops. Each cycle of selection fixes alleles and removes variation. This "genetic erosion" makes crops vulnerable to new diseases and climate shifts.

Reference [[Robustness and Evolvability]] -- genetic diversity is the raw material for future adaptation.

Reference [[Evolutionary Tinkering]] -- evolution works with existing variation. No variation = no tinkering = no adaptation.

### Diversity across gene pools

In [ ]:
# Diversity across geographic groups using real Arabidopsis data
# Compare allele frequency spectra between geographic groups

groups_unique = pheno_df['geographic_group'].unique()
group_he = {}

fig, axes = plt.subplots(1, len(groups_unique), figsize=(4 * len(groups_unique), 4), sharey=True)

for ax, group in zip(axes, groups_unique):
    group_ids = pheno_df[pheno_df['geographic_group'] == group].index
    group_ids = group_ids.intersection(snp_df.index)
    group_geno = snp_df.loc[group_ids]

    # Allele frequencies for this group
    group_freqs = group_geno.mean() / 2
    he = np.mean(2 * group_freqs * (1 - group_freqs))
    group_he[group] = he

    ax.hist(group_freqs, bins=20, range=(0, 1), color='#66BB6A', edgecolor='black', alpha=0.7)
    ax.set_title(f'{group}\nn={len(group_ids)}, He={he:.4f}')
    ax.set_xlabel('Allele Frequency')
    if ax == axes[0]:
        ax.set_ylabel('Count')

plt.suptitle('Allele Frequency Spectra by Geographic Group (Real Arabidopsis Data)', fontsize=13)
plt.tight_layout()
plt.show()

# PCA to visualize population structure
pca = PCA(n_components=2)
pca_coords = pca.fit_transform(snp_df.loc[common_ids].values)

fig, ax = plt.subplots(figsize=(8, 6))
for group in groups_unique:
    mask = pheno_df.loc[common_ids, 'geographic_group'] == group
    ax.scatter(pca_coords[mask, 0], pca_coords[mask, 1], label=group, s=30, alpha=0.7)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_title('Population Structure (PCA on Real Arabidopsis SNPs)')
ax.legend()
plt.tight_layout()
plt.show()

print("Expected heterozygosity by group:")
for group, he in sorted(group_he.items(), key=lambda x: -x[1]):
    print(f"  {group}: He = {he:.4f}")

### Genetic erosion timeline

In [ ]:
# Genetic erosion timeline simulation
# Starting from real Arabidopsis diversity, model 20 breeding cycles
np.random.seed(42)
overall_he = np.mean(2 * (snp_df.mean() / 2) * (1 - snp_df.mean() / 2))

n_cycles = 20
diversity_loss_rate = 0.05  # 5% loss per cycle
he_timeline = [overall_he]

# Track diversity with occasional wild germplasm introduction
for cycle in range(1, n_cycles + 1):
    current_he = he_timeline[-1] * (1 - diversity_loss_rate)
    # Wild germplasm introduction events at cycles 7 and 14
    if cycle in [7, 14]:
        current_he = current_he * 0.7 + overall_he * 0.3  # 30% recovery from wild
    he_timeline.append(current_he)

# Critical threshold
critical_threshold = overall_he * 0.3

fig, ax = plt.subplots(figsize=(10, 5))
cycles = range(n_cycles + 1)
ax.plot(cycles, he_timeline, 'o-', color='#F44336', markersize=6, label='Expected He')
ax.axhline(critical_threshold, color='red', linestyle='--', alpha=0.5,
          label=f'Critical threshold ({critical_threshold:.4f})')
ax.axhline(overall_he, color='#2196F3', linestyle=':', alpha=0.5,
          label=f'Wild diversity ({overall_he:.4f})')

# Mark introduction events
for cycle in [7, 14]:
    ax.annotate('Wild germplasm\nintroduction', xy=(cycle, he_timeline[cycle]),
               xytext=(cycle + 1, he_timeline[cycle] + 0.01),
               arrowprops=dict(arrowstyle='->', color='green'),
               fontsize=8, color='green')

ax.set_xlabel('Breeding Cycle')
ax.set_ylabel('Expected Heterozygosity (He)')
ax.set_title('Genetic Erosion Through Breeding Cycles')
ax.legend()
ax.set_xlim(-0.5, n_cycles + 0.5)

plt.tight_layout()
plt.show()

print(f"Starting He (real Arabidopsis): {overall_he:.4f}")
print(f"He after {n_cycles} cycles: {he_timeline[-1]:.4f}")
print(f"Total diversity loss: {100*(1 - he_timeline[-1]/overall_he):.1f}%")
print(f"Without introductions would be: {overall_he * (1 - diversity_loss_rate)**n_cycles:.4f}")
print(f"Germplasm introductions preserved {100*(he_timeline[-1] - overall_he * (1-diversity_loss_rate)**n_cycles) / overall_he:.1f}% of diversity")

---

## Summary

| Concept | What you built | Why it matters |
|---------|---------------|----------------|
| Polyploidy | Genome size comparison | Plants break diploid rules |
| TE content | Genome composition | Most plant DNA is mobile elements |
| Gene duplication | WGD + fractionation model | [[Degeneracy in Biological Systems]] |
| Domestication | Bottleneck + sweep simulation | How wild plants became crops |
| Crop GWAS | Yield association mapping | Find genes for breeding |
| Genomic prediction | Ridge regression on markers | Modern breeding tool (GS) |
| GxE interaction | Reaction norms + stability | [[Context Conditionality]] in agriculture |
| Conservation | Diversity erosion | Wild relatives = future food security |

**Connections to conceptual framework:**
- [[Degeneracy in Biological Systems]]: Polyploidy creates redundancy for evolutionary exploration
- [[Fitness Landscapes]]: Domestication navigates landscapes under human objectives
- [[Variant-Phenotype Mapping]]: Crop GWAS maps genotype to yield
- [[Genotype-to-Phenotype Hub]]: The central challenge across all species
- [[Context Conditionality]]: GxE interaction -- same genome, different environments, different yields
- [[Robustness and Evolvability]]: Genetic diversity enables future adaptation
- [[Evolutionary Tinkering]]: Evolution works with existing variation
- [[Plant-Animal Divergence]]: Why plant genomes are so different

**The complete series:**
- [[01_Sequence_Analysis_Fundamentals]] -- Biopython, the Central Dogma
- [[02_Genomic_Variant_Analysis]] -- Population genetics, GWAS
- [[03_Single_Cell_Transcriptomics]] -- scanpy, cell type discovery
- [[04_Protein_Structure_Drug_Discovery]] -- Structure and drug design
- [[05_Bulk_RNAseq_Differential_Expression]] -- Differential expression analysis
- [[06_Clinical_Biomedical_Informatics]] -- Clinical data and survival analysis
- [[07_Biomedical_Image_Analysis]] -- Pathology and radiology
- [[08_Plant_Biology_Agricultural_Genomics]] -- Crops and food security (this notebook)